# Text Augmentation

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/augmentation](https://github.com/huseinzol05/Malaya/tree/master/example/augmentation).
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
%%time

import malaya

/home/husein/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


CPU times: user 3.09 s, sys: 3.49 s, total: 6.58 s
Wall time: 2.05 s


### Why augmentation

Let say you have a very limited labelled corpus, and you want to add more, but labelling is very costly.

So, text augmentation! We provided few augmentation interfaces in Malaya.

### Load Synonym

Use dictionary of synonym to replace words with it synonyms. Synonym data from [Malaya-Dataset/90k-synonym](https://github.com/huseinzol05/Malaya-Dataset#90k-synonym).

```python
def synonym(
    string: str,
    threshold: float = 0.5,
    top_n = 5,
    **kwargs
):
    """
    augmenting a string using synonym, https://github.com/huseinzol05/Malaya-Dataset#90k-synonym

    Parameters
    ----------
    string: str
        this string input assumed been properly tokenized and cleaned.
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.

    Returns
    -------
    result: List[str]
    """
```

In [3]:
string = 'saya suka makan ayam dan ikan'
text = 'Perdana Menteri berkata, beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut. Bagaimanapun, beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik.'

In [4]:
tokenizer = malaya.preprocessing.Tokenizer()

In [5]:
malaya.augmentation.synonym(' '.join(tokenizer.tokenize(string)))

['saya suka mensyaratkan burung burung dan ikan',
 'saya menyukai mensyaratkan ayam dan ikan',
 'saya menghendaki memerlukan ayam dan ikan',
 'saya menghendaki memerlukan ayam dan ikan',
 'saya menghendaki perlu ayam jantan dan ikan']

In [6]:
malaya.augmentation.synonym(' '.join(tokenizer.tokenize(text)))

['Utama Menteri menunjukkan , beliau perlu memuat Maklumat terperinci berhubung isu berkenaan sebelum pentadbiran menggunakannya menaiki sebarang tindakan lanjut . Bagaimanapun , beliau keruan kesukaran itu dapat diselesaikan dan kerajaan kerajaan berupaya bertindak dengan cikgu .',
 'Utama Menteri menunjukkan , beliau perlu memuat Maklumat terperinci mencakup hal berkenaan sebelum pengurusan mempergunakan berjalan sebarang prosiding bekas . Bagaimanapun , beliau yakin kesukaran itu berupaya diselesaikan dan alam kerajaan dapat bergeser dengan guru .',
 'Utama Menteri menunjukkan , beliau berkehendakkan memuat Maklumat terperinci menyusahkan perhatian berkenaan sebelum penyelenggaraan menggunakannya berjalan sebarang tindakan bekas . Bagaimanapun , beliau percaya kesukaran itu berupaya diselesaikan dan kewujudan kerajaan dihukum beralih dengan guru .',
 'Utama Menteri menunjukkan , beliau mendesak memuat Maklumat terperinci menyusahkan perhatian berkenaan sebelum penjagaan menggunakann

### Load Wordvector

dictionary of synonym is quite hard to populate, required some domain experts to help us. So we can use wordvector to find nearest words.

```python
def wordvector(
    string: str,
    wordvector,
    threshold: float = 0.5,
    top_n: int = 5,
    soft: bool = False,
):
    """
    augmenting a string using wordvector.

    Parameters
    ----------
    string: str
    wordvector: object
        wordvector interface object.
    threshold: float, optional (default=0.5)
        random selection for a word.
    soft: bool, optional (default=False)
        if True, a word not in the dictionary will be replaced with nearest jarowrinkler ratio.
        if False, it will throw an exception if a word not in the dictionary.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.

    Returns
    -------
    result: List[str]
    """
```

In [7]:
vocab_wiki, embedded_wiki = malaya.wordvector.load(model = 'wikipedia')
word_vector_wiki = malaya.wordvector.WordVector(embedded_wiki, vocab_wiki)

Load pretrained wordvector into `malaya.wordvector.WordVector` class will disable eager execution.
2022-09-15 16:57:05.776615: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-15 16:57:05.780391: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-15 16:57:05.780415: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-09-15 16:57:05.780418: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-09-15 16:57:05.780486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was una

In [8]:
malaya.augmentation.wordvector(
    ' '.join(tokenizer.tokenize(string)), word_vector_wiki, soft = True
)

['saya suka makan ayam dan ikan',
 'kamu gemar minum ayam dan ikan',
 'anda pandai tidur ayam dan ikan',
 'kami senang mandi ayam dan ikan',
 'aku ingin berehat ayam dan ikan']

In [9]:
malaya.augmentation.wordvector(
    ' '.join(tokenizer.tokenize(text)), word_vector_wiki, soft = True
)

['Perdana Menteri berkata , beliau perlu memperoleh maklumat terperinci berhubung isu berkenaan sebelum kerajaan dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau yakin masalah itu dapat diselesaikan dan pentadbiran kerajaan boleh berfungsi dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh data mendalam berhubung isu berkenaan sebelum kerajaan boleh mendapat sebarang dakwaan terperinci . Bagaimanapun , dia yakin masalah tersebut dapat diselesaikan dan pemerintahan pemerintah boleh dikelaskan dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh bacaan menyeluruh berhubung isu berkenaan sebelum kerajaan harus menghabiskan sebarang kesalahan lanjutan . Bagaimanapun , baginda yakin masalah ini dapat diselesaikan dan perancangan perlembagaan boleh bertindak dengan baik .',
 'Perdana Menteri berkata , beliau perlu memperoleh penjelasan efektif berhubung isu berkenaan sebelum kerajaan perlu mengubah sebarang perbuatan ringkas . Bagaimanapun , me

### Load Transformer

Problem with wordvector, it just replaced a word for near synonym without understood the whole sentence context, so, Transformer comes to the rescue!

```python
def transformer(
    string: str,
    model,
    threshold: float = 0.5,
    top_p: float = 0.9,
    top_k: int = 100,
    temperature: float = 1.0,
    top_n: int = 5,
):

    """
    augmenting a string using transformer + nucleus sampling / top-k sampling.

    Parameters
    ----------
    string: str
    model: object
        transformer interface object. Right now only supported BERT, ALBERT and ELECTRA.
    threshold: float, optional (default=0.5)
        random selection for a word.
    top_p: float, optional (default=0.8)
        cumulative sum of probabilities to sample a word. 
        If top_n bigger than 0, the model will use nucleus sampling, else top-k sampling.
    top_k: int, optional (default=100)
        k for top-k sampling.
    temperature: float, optional (default=0.8)
        logits * temperature.
    top_n: int, (default=5)
        number of nearest neighbors returned. Length of returned result should as top_n.

    Returns
    -------
    result: List[str]
    """
```

In [10]:
electra = malaya.transformer.load(model = 'electra')

Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /home/husein/Malaya/electra-model/base/electra-base/model.ckpt


In [12]:
malaya.augmentation.transformer(' '.join(tokenizer.tokenize(string)), electra)

['saya nak makan nasi dan ikan',
 'saya terpaksa makan ikan / ikan',
 'saya kurang makan babi sama ikan',
 'saya lg makan opor dan ikan',
 'saya tak makan ikan apa ikan']

In [11]:
malaya.augmentation.transformer(' '.join(tokenizer.tokenize(text)), electra)

2022-09-15 16:57:09.906859: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "103" frequency: 2112 num_cores: 20 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.3.90" } l1_cache_size: 49152 l2_cache_size: 1310720 l3_cache_size: 26214400 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


['Perdana Menteri berkata , kerajaan akan memperoleh maklumat lanjut berhubung isu berkenaan sebelum rakyat dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau berharap masalah itu dapat diselesaikan dan mendedahkan kerajaan boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , pihaknya dapat memperoleh maklumat baharu berhubung isu ini sebelum beliau dapat mengambil sebarang tindakan lanjut . Bagaimanapun , beliau berharap masalah itu dapat diselesaikan dan kaedah kerajaan boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , beliau telah memperoleh maklumat terperinci berhubung isu itu sebelum kerajaan dapat melihat sebarang tindakan lanjut . Bagaimanapun , beliau yakin masalah itu dapat diselesaikan apabila memastikan kerajaan boleh diselesaikan dengan baik .',
 'Perdana Menteri berkata , kerajaan perlu memperoleh maklumat lanjut berhubung isu berkenaan sebelum beliau dapat melihat sebarang tindakan lanjut . Bagaimanapun , beliau berharap masalah itu d